In [ ]:
!pip install cellpose

In [ ]:
import cellpose, os, cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
from cellpose import models, plot
from tqdm import tqdm


# rough cropper
def crop_microscope_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    x,y,w,h = cv2.boundingRect(cv2.threshold(gray,50,255,cv2.THRESH_BINARY)[1])
    crop = image[y:y+h, x:x+w]

    mask = np.zeros(crop.shape[:2], np.uint8)
    radius = max(h,w)//2
    cv2.circle(mask, (w//2, h//2), radius, 255, -1)

    circle = cv2.bitwise_and(crop, crop, mask=mask)
    return circle


if __name__ == "__main__":
  drive.mount('/content/drive', force_remount=True)

  model = models.CellposeModel(gpu=True)

  input_directory  = "/content/drive/MyDrive/MNT/CellposeSAM/Input"
  output_directory = "/content/drive/MyDrive/MNT/CellposeSAM/Output"

  cell_counts = []

  for i, fname in enumerate(tqdm(sorted(os.listdir(input_directory)), desc="Processing"), start=1):
      image_path = os.path.join(input_directory, fname)
      image = crop_microscope_image(image_path)
      masks, flows, styles = model.eval(image)

      # show masks to check if cellpose is working properly
      flowi = flows[0]
      fig = plt.figure(figsize=(12,5))
      plot.show_segmentation(fig, image, masks, flowi)
      plt.tight_layout()
      plt.show()

      cell_count = len(np.unique(masks)) - 1
      cell_counts.append((i, cell_count)) # saves as a tuple (image, cell_count)


  output_path = os.path.join(output_directory, "cell_counts.txt")
  with open(output_path, "w") as f:
      for image, cell_count in cell_counts:
          f.write(f"{image} {cell_count}\n")